In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving sherlock-holm.es_stories_plain-text_advs.txt to sherlock-holm.es_stories_plain-text_advs.txt


In [ ]:
# read the text file
with open(path_to_file, 'r', encoding='utf-8') as file:
    text = file.read()

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# now lets tockenize the text to create a sequence of words

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In the above code, teh text is tockenized, which means it is divided into individual words or tocken. The 'Tokenizer' object is created, which will handel the tockenization process. The 'fit_on_text' method of the tockenizer is called, passing the 'text' as input. This method analyise the text and building a avoculabery of unique words

In [ ]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

n the above code, yhe text data is splite into lines using te '\n' character as delimiter. For each line in the text, the ,
'text_to_sequence' method of the  tokenizer is used to  convert the line into a sequence of numerical tockens based on the previously created vocabulary. The resulting tocken list in th iterated over a for loop. For each iteration, a sub-sequence, or n-gram,  of tocken is extracted

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In  the above code input sequences are padded to ensure all sequences have the same length. The variable 'max_sequence_len' is assigned the maximum length among all the input sequences. The 'pad_sequences'  function is used to add or truncate the input sequences to match thois maximum length.

The 'pad_sequence' function takes the input_sequence list, sets the maximum length to 'max_sequence_len', and specifies that the padding should be added at the beginning of the each sequence using the 'padding=pre' argument. Finally, the input sequence are converted into a numpy array to facilate futher processing.

Now lets split the sequence into input and output

In [ ]:
x = input_sequences[:, :-1]
y = input_sequences[:, -1]

In the above code, the input sequence are split into two arrays ,'x' and 'y', to cretae the input and output for training the next_word prediction model. the 'X' array is assigned the values of all rows in the 'input_sequence' array except for last column. It means the x contains all the tockens in each sequence except

In [ ]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In the above code we are converting the output array into a suitable formate for training a model, where each target word is represented as a binary vector.

Now let's build a neural network architecture to train the model.

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


Now let us compile the training model

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x, y, epochs=100, verbose=1)

Epoch 1/100
3010/3010 [==============================] - 92s 30ms/step - loss: 6.2407 - accuracy: 0.0762
Epoch 2/100
3010/3010 [==============================] - 87s 29ms/step - loss: 5.5097 - accuracy: 0.1241
Epoch 3/100
3010/3010 [==============================] - 90s 30ms/step - loss: 5.1221 - accuracy: 0.1471
Epoch 4/100
3010/3010 [==============================] - 90s 30ms/step - loss: 4.7932 - accuracy: 0.1647
Epoch 5/100
3010/3010 [==============================] - 91s 30ms/step - loss: 4.4908 - accuracy: 0.1817
Epoch 6/100
3010/3010 [==============================] - 94s 31ms/step - loss: 4.2047 - accuracy: 0.2029
Epoch 7/100
3010/3010 [==============================] - 90s 30ms/step - loss: 3.9326 - accuracy: 0.2275
Epoch 8/100
3010/3010 [==============================] - 91s 30ms/step - loss: 3.6714 - accuracy: 0.2572
Epoch 9/100
3010/3010 [==============================] - 92s 30ms/step - loss: 3.4263 - accuracy: 0.2905
Epoch 10/100
3010/3010 [==============================]

In the above code, the model is being compiled and trained. The 'Compile' method configers the model for training. The 'loss' paramater is set to 'categorical_crossentropy', a commonly used loss function for multi class classification problems. The 'optimizer' paramater is set to 'adam', an optimization algorithm that adapts the learning rate during training.

The 'metrics' paramater is set 'accuracy' to moniter the accuracy during training. After compiling the model, the 'fit' method is also called to train the mdoel on the input sequence "X" and the corresponding output 'y'. The 'epochs' parameter specifies the number of times the training process will iterate over hte entire dataset. The 'verbose' paramater is set to '1' to display the training process.

The above code will take more than an hour to execute. Once this cod eis executed, here]'s how we can generate the next word predicting using our model.

In [ ]:
# seed_ text = "I will leave if they "
send_text = "I am"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([send_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    send_text += " " + output_word
print(send_text)

1/1 [==============================] - 0s 16ms/step
I am afraid that you
